In [5]:
#Import the libraries
import numpy as np
import pandas as pd

import pickle

import warnings
warnings.filterwarnings("ignore")

In [6]:
train_data = pickle.load(open('clean_dataset_2022/train_set.bin', 'rb'))

test_data = pickle.load(open('clean_dataset_2022/test_set.bin', 'rb'))

## SARIMAX

In [7]:
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

from matplotlib.pyplot import figure

### Resample

In [8]:
province = ['BKK','CNX','KKC','RAY','SARA','SURAT']

In [9]:
for p in province:
    train_data[p] = train_data[p].resample('6H').mean()

    test_data[p] = test_data[p].resample('6H').mean()

### Split 70% 30%

In [10]:
train_set = {} ; valid_set = {}

ratio = 0.7

for p in province:
    train_size, valid_size = int(ratio*train_data[p].shape[0]), int((1-ratio)*train_data[p].shape[0])
    train_set[p], valid_set[p] = train_data[p].iloc[:train_size], train_data[p].iloc[train_size: ]

### Tuning Parameters

In [133]:
order = (1, 0, 1)
seasonal_order = (0, 1, 0, 1461) # 365.25 * 4

exog_order = (5, 0, 4)
exog_seasonal_order = (1, 0, 1, 1461) # 365.25 * 4

exog_columns = ['Temp', 'WindSpeed', 'WindDir']

### Training 6 provinces with *minimal_SARIMAX*

In [83]:
from importlib import reload

In [114]:
from custom_function import minimalSARIMAX

reload(minimalSARIMAX)

from custom_function.minimalSARIMAX import MinimalSARIMAX

In [125]:
model = {}
model_exog = {}

for p in province:
    model[p] = MinimalSARIMAX(train_data[p][['PM25']],
                order,
                seasonal_order,
                exog=train_data[p][exog_columns])
    
    model_exog[p] = {}    
    for exog in exog_columns:
        model_exog[p][exog] = MinimalSARIMAX(train_data[p][[exog]],
                        exog_order,
                        exog_seasonal_order)

In [126]:
for p in province:
    model[p].fit(lr=1e-5, lr_decay=0.999 ,verbose=0)

100%|██████████| 2555/2555 [00:00<00:00, 9028.88it/s]


In [127]:
for p in province:
    for exog in exog_columns:
        model_exog[p][exog].fit(lr=1e-5, lr_decay=0.999 ,verbose=0)

100%|██████████| 2555/2555 [00:00<00:00, 8806.57it/s]


In [128]:
y_pred = {} ; Err = {}

In [131]:
prev_data = {}
prev_exog = {}

y_pred['BKK'], Err['BKK'] = model['BKK'].predict(test_data['BKK'][['PM25']], y_X=test_data['BKK'][exog_columns], verbose=0)

In [132]:
model['BKK'].RMSE(y_pred['BKK'], test_data['BKK'][['PM25']])

Test on SARIMAX with RMSE: 3.8980861762664767


In [22]:
df_test = valid_set['BKK'].copy()

In [ ]:
y_t = df_test.iloc[:,[0]].to_numpy().ravel()

y_Xt = df_test[exog_columns].to_numpy()

y_pred = [10]

Error = [y_t[0]-10]


verbose = 0

for t in range(1,len(y_t)):
    pred = {} ; x = {}
    pred['p'], x['p'] = model['BKK'].p_prediction(y_t, t)
    pred['q'], x['q'] = model['BKK'].q_prediction(y_t, Error, t)
    pred['pX'], x['pX'] = model['BKK'].pX_prediction(y_Xt, t)
    pred['P'], x['P'] = model['BKK'].P_prediction(y_t, t)
    pred['Q'], x['Q'] = model['BKK'].Q_prediction(y_t, Error, t)

    pred['y'] = (pred['p'] + pred['q'] + pred['P'] + pred['Q'] + model['BKK'].params['c']).sum()
    
    y_pred.append(pred['y'])

    error_t = y_t[t] - pred['y']

    if verbose:
        print(t, pred['y'], y_t[t], error_t)

    Error.append(error_t)

y_pred_tmp = df_test.iloc[:,[0]].copy()
y_pred_tmp['PM25'] = np.array(y_pred)

In [18]:
# cnx_data = pd.concat((cnx_train, cnx_valid, cnx_test), axis=0)
# bkk_data = pd.concat((bkk_train, bkk_valid, bkk_test), axis=0)

In [19]:
# model_cnx.plot(cnx_data['PM25'], cnx_y_pred['PM25'], "Chiangmai PM2.5 Prediction")

In [20]:
# model_bkk.plot(bkk_data['PM25'], bkk_y_pred['PM25'], "Bangkok PM2.5 Prediction")

In [21]:
# # Open file - Write binary mode
# model_file = open('mod_cnx[0-1-1_1-1-0-365].model', 'wb')

# # Save Decision tree model
# pickle.dump(mod_cnx, model_file)

# # Close file
# model_file.close()

In [22]:
# test_exog = pd.concat((cnx_valid[exog_columns], cnx_test[exog_columns]), axis=0)

In [23]:
# bkk_data = pd.concat((bkk_train, cnx_valid, bkk_test), axis=0)

# Grid Search

In [24]:
# from sklearn.model_selection import GridSearchCV

In [134]:
import itertools
import statsmodels.api as sm

In [135]:
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
pdq

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 2, 0),
 (2, 2, 1),
 (2, 2, 2)]

In [136]:
pdqs = [(x[0], x[1], x[2], 1461) for x in list(itertools.product(p, d, q))]
pdqs

[(0, 0, 0, 1461),
 (0, 0, 1, 1461),
 (0, 0, 2, 1461),
 (0, 1, 0, 1461),
 (0, 1, 1, 1461),
 (0, 1, 2, 1461),
 (0, 2, 0, 1461),
 (0, 2, 1, 1461),
 (0, 2, 2, 1461),
 (1, 0, 0, 1461),
 (1, 0, 1, 1461),
 (1, 0, 2, 1461),
 (1, 1, 0, 1461),
 (1, 1, 1, 1461),
 (1, 1, 2, 1461),
 (1, 2, 0, 1461),
 (1, 2, 1, 1461),
 (1, 2, 2, 1461),
 (2, 0, 0, 1461),
 (2, 0, 1, 1461),
 (2, 0, 2, 1461),
 (2, 1, 0, 1461),
 (2, 1, 1, 1461),
 (2, 1, 2, 1461),
 (2, 2, 0, 1461),
 (2, 2, 1, 1461),
 (2, 2, 2, 1461)]

In [210]:
# Define function
def sarimax_gridsearch(y_train, y_test, pdq, PDQs, x_train = None, x_test = None):
    '''
    Input: 
        pm_train: PM2.5 training data
        exo_train: exogenous training data
        pdq : ARIMA combinations 
        pdqs : seasonal ARIMA combinations 

    Return:
        Prints out top 5 parameter combinations
        Returns dataframe of parameter combinations ranked by RMSE
    '''
    ans = []
    for comb in pdq:
        for combs in PDQs:
            p, d, q = comb[0], comb[1], comb[2]
            P, D, Q = combs[0], combs[1], combs[2]
            if (p+q <= 2) and (d <= 2) and (d+D <= 2) and (P+Q <= 1):  
                try:
                    model = MinimalSARIMAX(y_train, comb, combs,exog=x_train)
                    model.fit(lr=1e-5, lr_decay=0.999 ,verbose=0) 
                    y_pred, err = model.predict(y_test, y_X=x_test, verbose=0)
                    rmse = model.scoring(y_pred, y_test)
                    ans.append([comb, combs, rmse])
                    # print(f'SARIMAX {comb} x {combs} : RMSE Calculated ={rmse}')
                except Exception as e: 
                    # print(e)
                    continue

    # Convert into a dataframe
    ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'rmse'])

    # Sort and return top 5 combinations
    ans_df = ans_df.sort_values(by=['rmse'],ascending=True)[0:5]
    
    return ans_df.iloc[0]

In [183]:
pm_train_bkk = train_data[province[0]][['PM25']]
pm_test_bkk = test_data[province[0]][['PM25']]
exo_train_bkk = train_data[province[0]][exog_columns]
exo_test_bkk = test_data[province[0]][exog_columns]

pm_train_cnx = train_data[province[1]][['PM25']]
pm_test_cnx = test_data[province[1]][['PM25']]
exo_train_cnx = train_data[province[1]][exog_columns]
exo_test_cnx = test_data[province[1]][exog_columns]

pm_train_kkc = train_data[province[2]][['PM25']]
pm_test_kkc = test_data[province[2]][['PM25']]
exo_train_kkc = train_data[province[2]][exog_columns]
exo_test_kkc = test_data[province[2]][exog_columns]

pm_train_ray = train_data[province[3]][['PM25']]
pm_test_ray = test_data[province[3]][['PM25']]
exo_train_ray = train_data[province[3]][exog_columns]
exo_test_ray = test_data[province[3]][exog_columns]

pm_train_sara = train_data[province[4]][['PM25']]
pm_test_sara = test_data[province[4]][['PM25']]
exo_train_sara = train_data[province[4]][exog_columns]
exo_test_sara = test_data[province[4]][exog_columns]

pm_train_surat = train_data[province[5]][['PM25']]
pm_test_surat = test_data[province[5]][['PM25']]
exo_train_surat = train_data[province[5]][exog_columns]
exo_test_surat = test_data[province[5]][exog_columns]

In [211]:
result_bkk = sarimax_gridsearch(pm_train_bkk, pm_test_bkk, pdq, pdqs, exo_train_bkk, exo_test_bkk)
result_bkk

  0%|          | 0/4383 [00:00<?, ?it/s]


pdq           (1, 0, 1)
pdqs    (0, 1, 1, 1461)
rmse           3.699535
Name: 27, dtype: object

In [212]:
result_cnx = sarimax_gridsearch(pm_train_cnx, pm_test_cnx, pdq, pdqs, exo_train_cnx, exo_test_cnx)
result_cnx

  0%|          | 0/4383 [00:00<?, ?it/s]


pdq           (1, 0, 1)
pdqs    (0, 1, 1, 1461)
rmse           9.954083
Name: 27, dtype: object

In [213]:
result_kkc = sarimax_gridsearch(pm_train_kkc, pm_test_kkc, pdq, pdqs, exo_train_kkc, exo_test_kkc)
result_kkc

  0%|          | 0/4383 [00:00<?, ?it/s]


pdq           (2, 1, 0)
pdqs    (0, 1, 1, 1461)
rmse           6.848745
Name: 51, dtype: object

In [214]:
result_ray = sarimax_gridsearch(pm_train_ray, pm_test_ray, pdq, pdqs, exo_train_ray, exo_test_ray)
result_ray

  0%|          | 0/4383 [00:00<?, ?it/s]


pdq           (2, 0, 0)
pdqs    (0, 1, 0, 1461)
rmse           4.543428
Name: 44, dtype: object

In [215]:
result_sara = sarimax_gridsearch(pm_train_sara, pm_test_sara, pdq, pdqs, exo_train_sara, exo_test_sara)
result_sara

  0%|          | 0/4383 [00:00<?, ?it/s]


pdq           (2, 0, 0)
pdqs    (0, 0, 1, 1461)
rmse           7.187353
Name: 22, dtype: object

In [216]:
result_surat = sarimax_gridsearch(pm_train_surat, pm_test_surat, pdq, pdqs, exo_train_surat, exo_test_surat)
result_surat

  0%|          | 0/2555 [00:00<?, ?it/s]


pdq           (1, 1, 1)
pdqs    (0, 1, 0, 1461)
rmse            5.74374
Name: 38, dtype: object

In [219]:
order = {province[0]: result_bkk.pdq, province[1]: result_cnx.pdq, province[2]: result_kkc.pdq, province[3]: result_ray.pdq, province[4]: result_sara.pdq, province[5]: result_surat.pdq}
seasonal_order = {province[0]: result_bkk.pdqs, province[1]: result_cnx.pdqs, province[2]: result_kkc.pdqs, province[3]: result_ray.pdqs, province[4]: result_sara.pdqs, province[5]: result_surat.pdqs}
print(order)
print(seasonal_order)

{'BKK': (1, 0, 1), 'CNX': (1, 0, 1), 'KKC': (2, 1, 0), 'RAY': (2, 0, 0), 'SARA': (2, 0, 0), 'SURAT': (1, 1, 1)}
{'BKK': (0, 1, 1, 1461), 'CNX': (0, 1, 1, 1461), 'KKC': (0, 1, 1, 1461), 'RAY': (0, 1, 0, 1461), 'SARA': (0, 0, 1, 1461), 'SURAT': (0, 1, 0, 1461)}


In [ ]:
for exog in exog_columns:
        model_exog[p][exog] = MinimalSARIMAX(train_data[p][[exog]],
                        exog_order,
                        exog_seasonal_order)